In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import create_extraction_chain
import os
import sys
from dotenv import load_dotenv
import openai

load_dotenv()

openai.api_key = "5d050ffec2b94f5eb43c54c80149561e"#os.environ["OPENAI_API_KEY"]
openai.api_base = "https://futu-002-caeast-001.openai.azure.com/"#os.environ["OPENAI_API_BASE"]
openai.api_type = "azure"
openai.api_version = "2023-05-15"
# chat_deployment = os.environ["OPENAI_CHAT_DEPLOYMENT"]
# OPENAI_EMBEDDING_DEPLOYMENT = os.environ["OPENAI_EMBEDDING_DEPLOYMENT"]

In [3]:
import fitz

In [6]:
# project_dir = "/Users/jarvixwang/Documents/Project/gpt-researcher"
doc = fitz.open('../data/TSLA_Stock_Announcement_1.pdf')#os.path.join(project_dir, 'data/TSLA_Stock_Announcement_1.pdf'))

text = ""
for page in doc:
    text += "\n"+page.get_text()



In [9]:
# print(text)

In [10]:
# schema在后面代码中没用
schema = {
    "properties": {
        "action_status": {"type": "string"},
        "action_time": {"type": "integer"},
        "action_entity": {"type": "string"},
        "action": {"type": "string"},
        "nature_of_stocks": {"type": "string"},
        "number_of_stocks": {"type": "double"},
        "transaction_price": {"type": "double"},
        "total_value": {"type": "double"},
        "stocks_left_after_action": {"type": "double"},
    },
    "required": ["action_status", "action_time", "action_entity", "action", "nature_of_stocks", "number_of_stocks", "transaction_price", "total_value", "stocks_left_after_action"],
}

In [20]:
from typing import Sequence, Optional
from langchain.prompts import (
    PromptTemplate,
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)
from enum import Enum
from langchain.llms import OpenAI
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser

class StockOptions(Enum):
    """交易的股票类型"""
    rsu='Restricted Stock Unit'
    eso='Employee Stock Option'
    nso='Non-Qualified Stock Option'
    iso='Incentive Stock Option'
    psu='Performance Stock Unit'
    espp='Employee Stock Purchase Plan'
    dsu='Deferred Stock Unit'
    sars='Stock Appreciation Right'
    phantom='Phantom Stock'


class StockTransaction(BaseModel):
    """股票交易类公告需要提取的字段"""
    issuer_name: str
    transaction_status: str
    transaction_date: str
    transaction_entity: str
    transaction_entity_relationship_to_issuer: str
    buy_or_sell: str
    stock_options: Sequence[StockOptions]
    number_of_stocks_in_transaction: float
    transaction_unit_price: float
    transaction_total_value: float
    stocks_left_after_transaction: float
    transaction_currency: str
    

In [21]:
parser = PydanticOutputParser(pydantic_object=StockTransaction)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)


In [22]:
llm = AzureChatOpenAI(
            temperature=0.0,
            deployment_name='gpt-4',
            openai_api_key=openai.api_key,
            openai_api_base=openai.api_base,
            openai_api_type="azure",
            openai_api_version="2023-05-15",
        )

_input = prompt.format_prompt(query=text)
output = llm.predict(_input.to_string())
stock_transaction = parser.parse(output)

In [16]:
from IPython.display import display, JSON

In [23]:
display(JSON(stock_transaction.json()))

/var/folders/t0/wrmv7mvs6n9dwrlrnx0hp18c0000gn/T/ipykernel_84880/3711024533.py:1: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  display(JSON(stock_transaction.json()))
/Users/jarvixwang/miniconda3/envs/gpt-researcher/lib/python3.11/site-packages/IPython/core/display.py:618: UserWarning: JSON expects JSONable dict or list, not JSON strings
  warnings.warn("JSON expects JSONable dict or list, not JSON strings")


<IPython.core.display.JSON object>

In [31]:
# JSON(stock_transaction.json(),expanded=True)
import json
# json.dumps()
a  = json.dumps(json.loads(stock_transaction.json()), indent=4)
# print(type(a))
print(a)
# print(stock_transaction.json())

{
    "issuer_name": "Tesla, Inc.",
    "transaction_status": "LIVE",
    "transaction_date": "09/05/2023",
    "transaction_entity": "ZACHARY KIRKHORN",
    "transaction_entity_relationship_to_issuer": "Officer",
    "buy_or_sell": "Sell",
    "stock_options": [
        "Restricted Stock Unit"
    ],
    "number_of_stocks_in_transaction": 3752.0,
    "transaction_unit_price": 244.8,
    "transaction_total_value": 919240.0,
    "stocks_left_after_transaction": 3173994.0,
    "transaction_currency": "USD"
}


/var/folders/t0/wrmv7mvs6n9dwrlrnx0hp18c0000gn/T/ipykernel_84880/1793570594.py:4: PydanticDeprecatedSince20: The `json` method is deprecated; use `model_dump_json` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.4/migration/
  a  = json.dumps(json.loads(stock_transaction.json()), indent=4)
